# Imports

In [4]:
import requests, json
from pprint import pprint
import pandas as pd
import Player
from importlib import reload
reload(Player)
from Player import Player

In [5]:
pd.set_option('display.max_columns', None)

# URL 

In [6]:
r = requests.get('https://api.football-data.org/v4/matches')


In [7]:
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

## Players

In [8]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# Show id is the PK
print(players['id'].nunique())
print(len(players))

players = players.rename(columns={'id': 'player_id'})
players['player_name'] = players['first_name'] + ' ' + players['second_name']
# show some information about first five players
players[['player_id', 'player_name', 'team', 'element_type']].head()

741
741


,player_id,player_name,team,element_type
0,1,David Raya Martín,1,1
1,2,Kepa Arrizabalaga Revuelta,1,1
2,3,Karl Hein,1,1
3,4,Tommy Setford,1,1
4,5,Gabriel dos Santos Magalhães,1,2


## Teams

In [9]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])
teams = teams.rename(columns={'id': 'team_id'})

teams.sort_values(by='strength')

,code,draw,form,team_id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
2,90,0,None,3,0,Burnley,0,0,16,BUR,2,None,False,0,1050,1050,1050,1050,1050,1050,43
16,56,0,None,17,0,Sunderland,0,0,7,SUN,2,None,False,0,1050,1050,1050,1050,1050,1050,29
10,2,0,None,11,0,Leeds,0,0,12,LEE,2,None,False,0,1050,1075,1050,1050,1050,1100,9
9,54,0,None,10,0,Fulham,0,0,8,FUL,3,None,False,0,1125,1125,1130,1130,1120,1120,34
17,6,0,None,18,0,Spurs,0,0,3,TOT,3,None,False,0,1130,1175,1100,1100,1160,1250,21
15,17,0,None,16,0,Nott'm Forest,0,0,15,NFO,3,None,False,0,1165,1205,1150,1230,1180,1180,15
13,1,0,None,14,0,Man Utd,0,0,11,MUN,3,None,False,0,1105,1125,1110,1110,1100,1140,12
18,21,0,None,19,0,West Ham,0,0,19,WHU,3,None,False,0,1100,1100,1100,1100,1100,1100,25
19,39,0,None,20,0,Wolves,0,0,20,WOL,3,None,False,0,1100,1125,1080,1100,1120,1150,38
7,31,0,None,8,0,Crystal Palace,0,0,5,CRY,3,None,False,0,1140,1160,1120,1130,1160,1190,6


## Positions

In [10]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])
positions = positions.rename(columns={'id': 'position_id'})

positions

,position_id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],86
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],245
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],328
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],82


Merge team, positions and players

In [11]:
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='team_id'
)

df = df.merge(
    positions,
    left_on='element_type',
    right_on='position_id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
    ['first_name', 'second_name', 'team_name', 'position_name']
].head()

,first_name,second_name,team_name,position_name
0,David,Raya Martín,Arsenal,Goalkeeper
1,Kepa,Arrizabalaga Revuelta,Arsenal,Goalkeeper
2,Karl,Hein,Arsenal,Goalkeeper
3,Tommy,Setford,Arsenal,Goalkeeper
4,Gabriel,dos Santos Magalhães,Arsenal,Defender


In [ ]:
df['points_per_game']

In [30]:
df['points_per_game'] = df['points_per_game'].astype(float)

In [35]:
sub_df.sort_values(by='points_per_game', ascending=False)[['player_name', 'points_per_game', 'total_points']].head(20)

,player_name,points_per_game,total_points
472,Erling Haaland,9.2,46
134,Antoine Semenyo,8.2,41
244,Trevoh Chalobah,7.6,38
124,Marcos Senesi Barón,7.4,37
279,Marc Guéhi,7.2,36
266,João Pedro Junqueira de Jesus,7.0,35
6,Riccardo Calafiori,7.0,35
7,Jurriën Timber,7.0,35
430,Ryan Gravenberch,6.8,27
94,Jaidon Anthony,6.8,34


In [31]:
import optimisation
from importlib import reload
reload(optimisation)
from optimisation import TeamOptimizer
df['now_cost'] = df['now_cost']/10
sub_df = df[df['minutes']>180]

optimizer = TeamOptimizer(df, 'points_per_game')
optimizer.solve()

=== DATA FEASIBILITY CHECK ===
Available Goalkeeper: 86
Available Defender: 245
Available Midfielder: 328
Available Forward: 82
✅ Goalkeeper: 86 >= 2 (OK)
✅ Defender: 245 >= 5 (OK)
✅ Midfielder: 328 >= 5 (OK)
✅ Forward: 82 >= 3 (OK)
Min cost for 2 Goalkeeper players: 7.900000000000001e-11
Min cost for 5 Defender players: 1.95e-10
Min cost for 5 Midfielder players: 2.2000000000000002e-10
Min cost for 3 Forward players: 1.31e-10
Minimum possible team cost: 6.25e-10
Budget: 100
✅ Budget: 6.25e-10 <= 100 (OK)

=== SOLVING OPTIMIZATION ===
Players: 741
Budget: 100
Team size: 15
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/eoinmolloy/Documents/Documents/github/fantasy_premier_league/.venv/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/5g/y4dx7m714l7ft2p__s3ybpn80000gn/T/e209e3e7aa1d42fc8cf165945b1699d8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/5g/y4dx7m714l7ft2p__s3

{'squad': ['Riccardo Calafiori',
  'Jaidon Anthony',
  'Marcos Senesi Barón',
  'Antoine Semenyo',
  'Solly March',
  'Trevoh Chalobah',
  'Josh Acheampong',
  'João Pedro Junqueira de Jesus',
  'Marc Guéhi',
  'Ryan Gravenberch',
  'Oscar Bobb',
  'Erling Haaland',
  'Senne Lammens',
  'Nick Pope',
  'Nazarii Rusyn'],
 'starters': ['Riccardo Calafiori',
  'Jaidon Anthony',
  'Marcos Senesi Barón',
  'Antoine Semenyo',
  'Trevoh Chalobah',
  'Josh Acheampong',
  'João Pedro Junqueira de Jesus',
  'Marc Guéhi',
  'Ryan Gravenberch',
  'Erling Haaland',
  'Nick Pope'],
 'formation': '5-3-2',
 'captain': 'Erling Haaland',
 'vice_captain': 'Trevoh Chalobah',
 'total_cost': np.float64(9.090000000000001e-10),
 'total_points': np.float64(91.80000000000001)}

# Players Gameweek History

In [22]:
def get_gameweek_history(player_id, players_df = None):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()

    if players_df is not None:
        player = players_df[players_df['player_id']==player_id]['player_name'].values[0]
        print('Gameweek history for ', player)
    
    # extract 'history' data from response into dataframe

    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
gameweek_res = get_gameweek_history(1, players_df=df)
gameweek_res

Gameweek history for  David Raya Martín


,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,9,14,10,False,2025-08-17T15:30:00Z,0,1,1,False,90,0,0,1,0,0,0,0,1,0,7,3,38,49.2,0.0,0.0,4.9,1,13,0,0,1,0.00,0.00,0.00,1.52,55,0,1531911,0,0
1,1,11,11,6,True,2025-08-23T16:30:00Z,5,0,2,False,90,0,0,1,0,0,0,0,0,0,1,0,28,13.4,0.0,0.0,1.3,0,3,0,0,1,0.00,0.00,0.00,0.17,55,218659,2284634,277339,58680
2,1,25,12,2,False,2025-08-31T15:30:00Z,1,0,3,False,90,0,0,0,1,0,0,0,0,0,2,0,12,20.0,10.0,0.0,3.0,0,12,0,0,1,0.00,0.02,0.02,0.52,55,-12311,2406964,146739,159050
3,1,31,16,6,True,2025-09-13T11:30:00Z,3,0,4,False,90,0,0,1,0,0,0,0,0,0,1,0,24,12.8,0.0,0.0,1.3,0,9,0,0,1,0.00,0.00,0.00,0.20,55,171289,2765759,289041,117752
4,1,41,13,2,True,2025-09-21T15:30:00Z,1,1,5,False,90,0,0,0,1,0,0,0,0,0,2,0,13,21.4,0.0,0.0,2.1,1,6,0,0,1,0.00,0.01,0.01,0.89,55,-9786,2762632,98100,107886


In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

players = {}
df_list = []

def fetch_player(row):
    player = Player(player_id=row.player_id, merged_df=df)
    player.get_player_gameweek_history()

    player_df = player.gameweek_history.copy()
    player_df["player_id"] = row.player_id
    player_df["player_name"] = row.player_name

    return row.player_name, player, player_df

with ThreadPoolExecutor(max_workers=20) as executor:  # tune worker count
    futures = [executor.submit(fetch_player, row) for row in df.itertuples(index=False)]
    for f in as_completed(futures):
        player_name, player, player_df = f.result()
        players[player_name] = player
        df_list.append(player_df)

player_gw_hist = pd.concat(df_list, ignore_index=True)


Initialised Initialised  Kepa Arrizabalaga Revuelta
 David Raya Martín
Initialised  Karl Hein
Initialised  Tommy Setford
Initialised  Gabriel dos Santos Magalhães
Initialised  William Saliba
Initialised  Riccardo Calafiori
Initialised  Jurriën Timber
Initialised  Jakub Kiwior
Initialised  Myles Lewis-Skelly
Initialised  Benjamin White
Initialised  Brayden Clarke
Initialised  Maldini Kacurri
Initialised  Josh Nichols
Initialised  Martin Ødegaard
Initialised  Noni Madueke
Initialised  Gabriel Martinelli Silva
Initialised  Leandro Trossard
Initialised  Bukayo Saka
Initialised  Declan Rice
Gameweek history for  Karl Hein
Initialised  Mikel Merino Zazón
Gameweek history for  David Raya Martín
Gameweek history for  Jakub Kiwior
Initialised  Fábio Ferreira Vieira
Initialised  Christian Nørgaard
Gameweek history for  Riccardo Calafiori
Initialised  Ethan Nwaneri
Gameweek history for  Jurriën Timber
Initialised  Martín Zubimendi Ibáñez
Gameweek history for  William Saliba
Initialised  Ismeal Ka

In [24]:
player_gw_hist['round'].unique()

array([1, 2, 3, 4, 5])

In [25]:
import plotly.express as px

# 1. Find top top_n players by total selections
top_n = 10
top_players = (
    player_gw_hist.groupby("player_name")["selected"]
    .sum()
    .nlargest(top_n)
    .index
)

# 2. Filter only those players
plot_df = player_gw_hist[player_gw_hist["player_name"].isin(top_players)]

# 3. Interactive line plot
fig = px.line(
    plot_df,
    x="round",
    y="selected",
    color="player_name",
    markers=True,
    title=f"Top {top_n} Players by Overall Selected Count Across Gameweeks",
    labels={"round": "Gameweek", "selected": "Selected (Number of Managers)"}
)

# Make legend scrollable if too many players
fig.update_layout(
    legend=dict(
        title="Player",
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.05
    ),
    width=1000,
    height=600
)

fig.show()


## Season history for a player

In [26]:
def get_season_history(player_id, players_df = None):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    if players_df is not None:
        player = players_df[players_df['player_id']==player_id]['player_name'].values[0]
        print('Gameweek history for ', player)
    

    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
season_res = get_season_history(1, players_df=df)
season_res

Gameweek history for  David Raya Martín


,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,clearances_blocks_interceptions,recoveries,tackles,defensive_contribution,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2021/22,154561,45,44,95,2160,0,0,8,27,0,0,0,1,0,78,5,496,593.4,10.0,0.0,60.1,0,0,0,0,0,0.00,0.00,0.00,0.00
1,2022/23,154561,45,48,166,3420,0,0,12,46,1,0,0,1,0,154,20,822,1146.0,20.1,4.0,117.3,0,0,0,0,38,0.11,0.12,0.23,50.12
2,2023/24,154561,50,53,135,2880,0,0,16,24,0,1,0,2,0,46,6,569,413.0,0.0,0.0,41.1,0,0,0,0,32,0.00,0.04,0.04,22.51
3,2024/25,154561,55,56,142,3420,0,0,13,34,0,0,0,3,0,86,10,555,755.4,10.7,0.0,76.7,0,0,0,0,38,0.00,0.03,0.03,35.03


Strategies
1. Simple expected points calculation
2. Following the trend of other players
3. Machine learning prediction